# 📚 AuraDB + Neo4j (Python) — Carga de un CSV de Películas y Modelo de Grafos

Este notebook guía paso a paso a tus alumnos para:
- Conectarse a **Neo4j AuraDB** desde Python.
- Crear **constraints** e **índices**.
- Definir **nodos** y **relaciones** (Movie, Year, Decade, bandas y Keywords).
- **Importar** un CSV local y **normalizar** datos.
- Ejecutar consultas de verificación y explorar el grafo.

## 🎯 Objetivos de aprendizaje
- Entender las credenciales necesarias para conectar con **AuraDB** (URI + usuario + contraseña).
- Crear y comprender **constraints** e **índices** (incluyendo **full‑text**).
- Cargar datos desde CSV **sin** `LOAD CSV` (usando `UNWIND $rows`) para no depender del directorio `/import`.
- Construir un pequeño **modelo de grafo** con nodos `:Movie`, `:Year`, `:Decade` y nodos de **bandas** (`:RatingBand`, `:RuntimeBand`, `:BoxOfficeBand`) además de `:Keyword`.
- Practicar **consultas Cypher** para validar el modelo.

## ✅ Requisitos previos
- Crear una cuenta gratuita de **Neo4j AuraDB** (https://neo4j.com/product/auradb/)
- Una instancia activa de **Neo4j AuraDB**.Con la cuenta gratuita solo se nos permite crear una.
- Las credenciales de conexión (⚠️ **no** es un “API key”): `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD`. Estos valores están dentro del archivo descargado. Solo copia y pega en tu .env
- Un **CSV local** con: `Movie Name`, `Year of Release`, `Watch Time`, `Movie Rating`, `Meatscore of movie`, `Votes`, `Gross`, `Description`.

> ℹ️ **API keys de Aura**: sirven para la **gestión** de instancias; **no** para ejecutar consultas Cypher. Para conectarte a la base necesitas **URI, usuario y contraseña** (se copian desde **Connect** en la consola de AuraDB).

## 1) Instalación de librerías

In [ ]:
%pip install -q python-dotenv pandas langchain-community neo4j ipywidgets

## 2) Configuración de credenciales y conexión
Puedes usar un fichero `.env` en el mismo directorio con:
```env
NEO4J_URI=neo4j+s://<tu-host>.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=<tu-contraseña>
CSV_PATH=top_1000_IMDB_movies_utf8.csv
```
O asignarlas directamente en variables. **No compartas** tu contraseña.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

NEO4J_URI      = os.getenv("NEO4J_URI") or "neo4j+s://<tu-host>.databases.neo4j.io"
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") or "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "<tu-contraseña>"

# Ruta al CSV local (cámbiala si es necesario)
CSV_PATH = os.getenv("CSV_PATH") or "top_1000_IMDB_movies_utf8.csv"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# Prueba rápida de conexión
graph.query("RETURN 1 AS ok")

## 3) (Opcional) Subir el CSV con un selector de archivos
Si trabajas en **Colab/Jupyter en la nube**, este widget te permite **subir** el CSV desde tu ordenador.
Si ya tienes el archivo en disco y conoces la ruta, **puedes saltarte esta sección** y ajustar `CSV_PATH` arriba.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(accept='.csv', multiple=False)
display(uploader)

def save_upload_to(path="uploaded.csv"):
    if not uploader.value:
        raise ValueError("Sube un CSV primero desde el widget.")
    (fname, fileinfo), = uploader.value.items()
    with open(path, "wb") as f:
        f.write(fileinfo["content"])
    return path

# Cuando termines:
# CSV_PATH = save_upload_to("top_1000_IMDB_movies_utf8.csv")
# CSV_PATH

## 4) Esquema: constraints e índices (full‑text incluido)

In [ ]:
schema_queries = [
    """CREATE CONSTRAINT movie_id IF NOT EXISTS
    FOR (m:Movie) REQUIRE m.movieId IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo movieId de los nodos Movie sea único
    """CREATE CONSTRAINT year_value IF NOT EXISTS
    FOR (y:Year) REQUIRE y.value IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo value de los nodos Year sea único
    """CREATE CONSTRAINT decade_value IF NOT EXISTS
    FOR (d:Decade) REQUIRE d.value IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo value de los nodos Decade sea único
    """CREATE CONSTRAINT ratingband_name IF NOT EXISTS
    FOR (r:RatingBand) REQUIRE r.name IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo name de los nodos RatingBand sea único
    """CREATE CONSTRAINT runtimeband_name IF NOT EXISTS
    FOR (r:RuntimeBand) REQUIRE r.name IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo name de los nodos RuntimeBand sea único
    """CREATE CONSTRAINT boxofficeband_name IF NOT EXISTS
    FOR (b:BoxOfficeBand) REQUIRE b.name IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo name de los nodos BoxOfficeBand sea único
    """CREATE CONSTRAINT keyword_name IF NOT EXISTS
    FOR (k:Keyword) REQUIRE k.name IS UNIQUE""".strip(),  # Crea un constraint para garantizar que el atributo name de los nodos Keyword sea único
    # Índices full-text (si ya existen, ignoramos el error)
    """CALL db.index.fulltext.createNodeIndex('movieFulltext', ['Movie'], ['title','description'])""".strip(),  # Crea un índice full-text para los nodos Movie en los atributos title y description
    """CALL db.index.fulltext.createNodeIndex('keywordFulltext', ['Keyword'], ['name'])""".strip()  # Crea un índice full-text para los nodos Keyword en el atributo name
]

for q in schema_queries:
    try:
        graph.query(q)  # Ejecuta cada consulta Cypher para crear constraints o índices
    except Exception:
        pass  # Ignora cualquier error que ocurra (por ejemplo, si el constraint ya existe)

# Imprime un mensaje indicando que el esquema ha sido creado o verificado
print("Esquema creado/verificado ✅")

## 5) Catálogos de bandas (Rating/Runtime/Box Office)

In [ ]:
# Definimos una lista de consultas Cypher para crear nodos de bandas de clasificación, duración y taquilla.
catalog_queries = [
    # Consulta para crear nodos de bandas de clasificación (RatingBand).
    """
    UNWIND [
      {name:'[9.0–10)', min:9.0, max:10.0},
      {name:'[8.5–9.0)', min:8.5, max:9.0},
      {name:'[8.0–8.5)', min:8.0, max:8.5},
      {name:'< 8.0',     min:0.0, max:8.0}
    ] AS b
    MERGE (rb:RatingBand {name:b.name})  # Crea o encuentra un nodo RatingBand con el nombre especificado.
    SET rb.min=b.min, rb.max=b.max       # Establece los valores mínimo y máximo para la banda.
    """.strip(),

    # Consulta para crear nodos de bandas de duración (RuntimeBand).
    """
    UNWIND [
      {name:'Short(<90)',     min:0,   max:90},
      {name:'Feature(90–150)',min:90,  max:150},
      {name:'Epic(>150)',     min:150, max:10000}
    ] AS r
    MERGE (rt:RuntimeBand {name:r.name})  # Crea o encuentra un nodo RuntimeBand con el nombre especificado.
    SET rt.min=r.min, rt.max=r.max        # Establece los valores mínimo y máximo para la banda.
    """.strip(),

    # Consulta para crear nodos de bandas de taquilla (BoxOfficeBand).
    """
    UNWIND [
      {name:'< $10M',     min:0.0,      max:1.0e7},
      {name:'$10–100M',   min:1.0e7,    max:1.0e8},
      {name:'$100–500M',  min:1.0e8,    max:5.0e8},
      {name:'≥ $500M',    min:5.0e8,    max:9.9e15}
    ] AS bb
    MERGE (b:BoxOfficeBand {name:bb.name})  # Crea o encuentra un nodo BoxOfficeBand con el nombre especificado.
    SET b.min=bb.min, b.max=bb.max          # Establece los valores mínimo y máximo para la banda.
    """.strip()
]

# Iteramos sobre cada consulta en la lista y la ejecutamos en la base de datos Neo4j.
for q in catalog_queries:
    graph.query(q)  # Ejecuta la consulta Cypher en la base de datos.

# Imprime un mensaje indicando que los catálogos de bandas han sido creados correctamente.
print("Catálogos de bandas listos ✅")

## 6) Utilidades de limpieza en Python

In [ ]:
import math, re, hashlib
import pandas as pd

# Función para analizar y convertir un año a un entero.
def parse_year(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None  # Verifica si el valor es nulo o NaN.
    s = str(x).strip().replace("(","").replace(")","")  # Limpia el valor eliminando paréntesis y espacios.
    return int(s) if s.isdigit() else None  # Convierte a entero si es un número válido.

# Función para analizar y convertir la duración de una película a minutos.
def parse_runtime_min(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None  # Verifica si el valor es nulo o NaN.
    s = str(x).strip().lower().replace(" min","")  # Limpia el valor eliminando "min" y espacios.
    return int(s) if s.isdigit() else None  # Convierte a entero si es un número válido.

# Función para analizar y convertir el número de votos a un entero.
def parse_votes(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None  # Verifica si el valor es nulo o NaN.
    s = re.sub(r"[^\d]","", str(x))  # Elimina caracteres no numéricos.
    return int(s) if s else None  # Convierte a entero si es un número válido.

# Función para analizar y convertir ingresos brutos (taquilla) a un valor en dólares.
def parse_gross_usd(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None  # Verifica si el valor es nulo o NaN.
    s = str(x).strip()  # Limpia el valor eliminando espacios.
    m = re.match(r"^\$?\s*([\d.,]+)\s*([MB])?$", s, re.IGNORECASE)  # Extrae el número y la unidad (M o B).
    if not m: return None  # Retorna None si no coincide con el patrón esperado.
    num = float(m.group(1).replace(",", ""))  # Convierte el número a flotante eliminando comas.
    unit = (m.group(2) or "").upper()  # Obtiene la unidad (M o B) en mayúsculas.
    if unit == "B": num *= 1_000_000_000  # Convierte de billones a dólares.
    elif unit == "M": num *= 1_000_000  # Convierte de millones a dólares.
    return float(num)  # Retorna el valor en dólares.

# Función para analizar y convertir el metascore a un entero.
def parse_metascore(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None  # Verifica si el valor es nulo o NaN.
    s = str(x).strip()  # Limpia el valor eliminando espacios.
    return int(s) if re.search(r"\d", s) and s.isdigit() else None  # Convierte a entero si es un número válido.

# Función para convertir un valor a flotante de forma segura.
def safe_float(x):
    try: return float(x)  # Intenta convertir el valor a flotante.
    except: return None  # Retorna None si ocurre un error.

# Función para generar un identificador único para una película basado en su título y año.
def movie_id(title, year):
    return hashlib.sha1(f"{title}|{year}".encode("utf-8")).hexdigest()  # Genera un hash SHA-1 único.

# Conjunto de palabras comunes que se deben excluir al extraer palabras clave.
STOPWORDS = set("""the and of a to in on by for with as is his her their from an at into over this that it its he she they them are was were be been or""".split())

# Función para extraer palabras clave de una descripción.
def extract_keywords(desc, min_len=4, max_kw=20):
    if not desc or not isinstance(desc, str): return []  # Verifica si la descripción es válida.
    clean = re.sub(r"[^A-Za-z\s]", " ", desc)  # Elimina caracteres no alfabéticos.
    toks = [t.lower() for t in clean.split()]  # Convierte las palabras a minúsculas.
    toks = [t for t in toks if len(t) >= min_len and t not in STOPWORDS]  # Filtra palabras según longitud y stopwords.
    out, seen = [], set()  # Inicializa listas para palabras clave y palabras vistas.
    for t in toks:
        if t not in seen:
            seen.add(t); out.append(t)  # Agrega palabras únicas a la lista de salida.
            if len(out) >= max_kw: break  # Detiene el proceso si se alcanza el máximo de palabras clave.
    return out  # Retorna la lista de palabras clave extraídas.

## 7) Leer el CSV y preparar registros

In [ ]:
# Cambia sep="," si tu archivo usa comas
df = pd.read_csv(CSV_PATH, sep=";", dtype=str, keep_default_na=False)

# Mapea cabeceras reales (ajusta si tu CSV difiere)
col_title     = "Movie Name"
col_year      = "Year of Release"
col_runtime   = "Watch Time"
col_imdb      = "Movie Rating"
col_metascore = "Meatscore of movie"
col_votes     = "Votes"
col_gross     = "Gross"
col_desc      = "Description"

df.head(3)

## 8) Construir el payload para Cypher (UNWIND $rows)

In [ ]:
# Inicializamos una lista vacía para almacenar los registros procesados.
records = []

# Iteramos sobre cada fila del DataFrame.
for _, row in df.iterrows():
    # Obtenemos y limpiamos el título de la película.
    title = (row.get(col_title) or "").strip()
    # Analizamos y convertimos el año de lanzamiento.
    year = parse_year(row.get(col_year))
    # Si el título o el año son inválidos, omitimos esta fila.
    if not title or year is None:
        continue

    # Creamos un diccionario para representar un registro de película.
    rec = {
        "movieId": movie_id(title, year),  # Generamos un identificador único para la película.
        "props": {  # Propiedades de la película.
            "title": title,  # Título de la película.
            "year": year,  # Año de lanzamiento.
            "runtimeMin": parse_runtime_min(row.get(col_runtime)),  # Duración en minutos.
            "imdbRating": safe_float(row.get(col_imdb)),  # Calificación IMDb.
            "metascore": parse_metascore(row.get(col_metascore)),  # Metascore.
            "votes": parse_votes(row.get(col_votes)),  # Número de votos.
            "grossUSD": parse_gross_usd(row.get(col_gross)),  # Ingresos brutos en dólares.
            "description": (row.get(col_desc) or "").strip(),  # Descripción de la película.
        },
        "keywords": extract_keywords((row.get(col_desc) or "").strip())  # Palabras clave extraídas de la descripción.
    }
    # Agregamos el registro procesado a la lista.
    records.append(rec)

# Imprimimos la cantidad de registros procesados.
len(records)

## 9) Upsert de `:Movie` por lotes (UNWIND $rows)

In [ ]:
# Definimos la consulta Cypher para insertar o actualizar nodos de tipo Movie.
UPSERT_MOVIES = """
UNWIND $rows AS row  # Desempaqueta cada fila del parámetro $rows.
MERGE (m:Movie {movieId: row.movieId})  # Busca o crea un nodo Movie con el identificador único movieId.
SET m += row.props  # Actualiza las propiedades del nodo con los valores proporcionados.
"""

# Definimos el tamaño del lote para procesar los registros en partes.
BATCH_SIZE = 500

# Iteramos sobre los registros en lotes del tamaño definido.
for i in range(0, len(records), BATCH_SIZE):
    batch = records[i:i+BATCH_SIZE]  # Extraemos un lote de registros.
    graph.query(UPSERT_MOVIES, params={"rows": batch})  # Ejecutamos la consulta Cypher con el lote actual.

# Imprimimos un mensaje indicando que las películas han sido insertadas o actualizadas correctamente.
print("Películas insertadas/actualizadas ✅")

## 10) Crear `:Year` y `:Decade` y enlazar bandas

In [ ]:
# Consulta Cypher para enlazar películas con nodos de año y década.
LINK_YEAR_DECADE = """
MATCH (m:Movie)  # Encuentra todos los nodos de tipo Movie.
MERGE (y:Year {value: m.year})  # Crea o encuentra un nodo Year con el valor del año de la película.
MERGE (d:Decade {value: (m.year/10)*10})  # Crea o encuentra un nodo Decade basado en la década del año.
MERGE (m)-[:RELEASED_IN]->(y)  # Crea una relación RELEASED_IN entre la película y el nodo Year.
MERGE (y)-[:IN_DECADE]->(d)  # Crea una relación IN_DECADE entre el nodo Year y el nodo Decade.
"""
graph.query(LINK_YEAR_DECADE)  # Ejecuta la consulta en la base de datos.

# Consulta Cypher para enlazar películas con bandas de calificación (RatingBand).
LINK_RATING = """
MATCH (m:Movie), (rb:RatingBand)  # Encuentra nodos Movie y RatingBand.
WHERE m.imdbRating IS NOT NULL AND m.imdbRating >= rb.min AND m.imdbRating < rb.max  # Filtra películas con calificación IMDb dentro del rango de la banda.
MERGE (m)-[:HAS_RATING_BAND]->(rb)  # Crea una relación HAS_RATING_BAND entre la película y la banda de calificación.
"""
graph.query(LINK_RATING)  # Ejecuta la consulta en la base de datos.

# Consulta Cypher para enlazar películas con bandas de duración (RuntimeBand).
LINK_RUNTIME = """
MATCH (m:Movie), (rt:RuntimeBand)  # Encuentra nodos Movie y RuntimeBand.
WHERE m.runtimeMin IS NOT NULL AND m.runtimeMin >= rt.min AND m.runtimeMin < rt.max  # Filtra películas con duración dentro del rango de la banda.
MERGE (m)-[:HAS_RUNTIME_BAND]->(rt)  # Crea una relación HAS_RUNTIME_BAND entre la película y la banda de duración.
"""
graph.query(LINK_RUNTIME)  # Ejecuta la consulta en la base de datos.

# Consulta Cypher para enlazar películas con bandas de taquilla (BoxOfficeBand).
LINK_BOXOFFICE = """
MATCH (m:Movie), (bb:BoxOfficeBand)  # Encuentra nodos Movie y BoxOfficeBand.
WHERE m.grossUSD IS NOT NULL AND m.grossUSD >= bb.min AND m.grossUSD < bb.max  # Filtra películas con ingresos brutos dentro del rango de la banda.
MERGE (m)-[:HAS_BOXOFFICE_BAND]->(bb)  # Crea una relación HAS_BOXOFFICE_BAND entre la película y la banda de taquilla.
"""
graph.query(LINK_BOXOFFICE)  # Ejecuta la consulta en la base de datos.

# Imprime un mensaje indicando que las dimensiones y enlaces han sido creados correctamente.
print("Dimensiones y enlaces listos ✅")

## 11) (Opcional) `:Keyword` y relaciones `HAS_KEYWORD`

In [ ]:
# Definimos la consulta Cypher para insertar o actualizar palabras clave y enlazarlas con películas.
UPSERT_KEYWORDS = """
UNWIND $rows AS row  # Desempaqueta cada fila del parámetro $rows.
MATCH (m:Movie {movieId: row.movieId})  # Busca el nodo Movie correspondiente al movieId proporcionado.
UNWIND row.keywords AS kw  # Desempaqueta cada palabra clave asociada a la película.
MERGE (k:Keyword {name: kw})  # Crea o encuentra un nodo Keyword con el nombre especificado.
MERGE (m)-[:HAS_KEYWORD]->(k)  # Crea una relación HAS_KEYWORD entre la película y la palabra clave.
"""

# Definimos el tamaño del lote para procesar los registros en partes.
BATCH_SIZE = 500

# Iteramos sobre los registros en lotes del tamaño definido.
for i in range(0, len(records), BATCH_SIZE):
    # Creamos un lote de palabras clave para las películas que tienen keywords.
    batch_kw = [{"movieId": r["movieId"], "keywords": r["keywords"]}
                for r in records[i:i+BATCH_SIZE] if r["keywords"]]
    # Si el lote no está vacío, ejecutamos la consulta Cypher con el lote actual.
    if batch_kw:
        graph.query(UPSERT_KEYWORDS, params={"rows": batch_kw})

# Imprimimos un mensaje indicando que las palabras clave han sido enlazadas correctamente.
print("Keywords enlazadas ✅")

## 12) Consultas de verificación

In [ ]:
graph.query("MATCH (m:Movie) RETURN count(m) AS movies")

In [ ]:
graph.query("""
MATCH (m:Movie)
RETURN m.title AS title, m.imdbRating AS rating
ORDER BY rating DESC, m.votes DESC
LIMIT 5
""")

In [ ]:
graph.query("""
MATCH (m:Movie)-[:RELEASED_IN]->(:Year)-[:IN_DECADE]->(d:Decade {value:1990})
RETURN m.title AS title, m.year AS year, m.imdbRating AS rating
ORDER BY rating DESC, m.votes DESC
LIMIT 10
""")

In [ ]:
graph.query("MATCH (k:Keyword) RETURN k.name AS keyword LIMIT 20")

## 13) Troubleshooting (errores frecuentes)
- **`AuthenticationFailed` / `Neo.ClientError.Security.CredentialsExpired`**: revisa usuario/contraseña en Aura; cambia la contraseña si es necesario y actualiza el `.env`.
- **`ServiceUnavailable` / `Failed to establish connection`**: revisa el **URI** (usa `neo4j+s://...`) y conectividad de tu red.
- **`Index already exists`**: normal si ejecutas el notebook más de una vez.
- **`Procedure not found: db.index.fulltext.createNodeIndex`**: usa Neo4j 5.x/Aura actual.
- Ajusta `parse_*` o nombres de columnas si cambia tu CSV.

## 14) Referencias y documentación oficial
- **Conexión a AuraDB (aplicaciones/Drivers):**  https://neo4j.com/docs/aura/connecting-applications/overview/
- **Neo4j Python Driver (manual):**  https://neo4j.com/docs/python-manual/current/
- **LangChain · `Neo4jGraph` (API):**  https://python.langchain.com/api_reference/community/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
- **Constraints (Cypher Manual):**  https://neo4j.com/docs/cypher-manual/current/constraints/
- **Full‑text indexes:**  https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/full-text-indexes/
- **`LOAD CSV` (alternativa):**  https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/
- **ipywidgets – `FileUpload`:**  https://ipywidgets.readthedocs.io/en/8.1.3/examples/Widget%20List.html#File-Upload
- **pandas – `read_csv`:**  https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

### (Opcional) Conexión con el driver oficial

In [ ]:
# from neo4j import GraphDatabase
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
# with driver.session() as session:
#     rec = session.run("RETURN 1 AS ok").single()
#     print(rec["ok"])  # 1
# driver.close()